In [1]:
import sys
sys.path.append('/Users/user/meepc')
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,RobustPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def calculate_fpr(y_actual, y_prediction):
    fp = 0
    tn = 0
    
    for i in range(len(y_actual)):
        if y_actual[i] == 0 and y_prediction[i] == 1:
            fp += 1
        elif y_actual[i] == 0 and y_prediction[i] == 0:
            tn += 1
    fpr= fp/(fp+tn)
    return fpr

In [3]:
hankel = Hankel()
corrhankel = Corrhankel() 
lag = 50
stride = 0.5


In [4]:
df1 = pd.read_csv('~/data/ctown/dataset03.csv')
df2 = pd.read_csv('~/data/ctown/dataset04.csv')

train_normal = pd.concat((df1,df2),axis=0,ignore_index=True)
train_attack = df2[df2['ATT_FLAG']==1]

In [5]:
sensors = [col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG', 'S_PU6','F_PU6','S_PU11','F_PU11']]

In [6]:
y_truth=train_normal.loc[:,'ATT_FLAG']
y_truth


0        0
1        0
2        0
3        0
4        0
        ..
12933    0
12934    0
12935    0
12936    0
12937    0
Name: ATT_FLAG, Length: 12938, dtype: int64

In [7]:
#attack indexes in original training data
temp = hankel.fit(np.array(y_truth) ,lag,stride)
attack_idx = np.any(temp>0,axis=0).astype(int)
attack_idx = np.where(attack_idx>0)[0]
attack_idx

array([418, 419, 420, 421, 430, 431, 432, 442, 443, 444, 445, 446, 462,
       463, 464, 465, 466, 467, 489, 490, 491, 492, 498, 499, 500, 501,
       502, 503, 506, 507, 508, 509, 510, 511], dtype=int64)

In [8]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))


In [9]:
#  train_attack[sensors]
X_attack = pd.DataFrame(index=train_attack.index, columns = sensors, data=scaler.fit_transform(train_attack[sensors])).reset_index().drop(columns=['index'])
X_attack

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J300,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422
0,0.510851,-0.546397,-0.878085,-2.084040,-1.526839,-2.239309,-0.533841,-0.623119,0.0,0.548804,...,-2.059407,0.794886,-2.054304,0.013617,-1.323365,0.506761,-1.344766,1.507526,0.763488,-2.086719
1,0.623110,-1.147382,-0.297260,-2.101038,-1.035659,-0.755638,-0.193180,-0.784230,0.0,0.506152,...,-2.298174,0.975545,-2.298500,0.105519,0.590949,1.095478,0.604112,-1.873753,0.980724,-2.343036
2,0.757819,-1.734552,0.341648,-1.710086,-0.312534,0.613905,0.182722,-0.896547,0.0,0.476418,...,-2.578631,0.907219,-2.577037,0.232052,0.696182,1.172992,0.708874,-1.758276,1.095377,-2.634306
3,0.735368,-1.858894,0.995076,-0.911185,0.451524,0.785098,0.770069,-0.477660,0.0,0.587312,...,-0.797353,1.370448,-0.783719,0.457146,0.419171,1.141594,0.433104,-1.461149,-0.877852,-0.618722
4,0.690464,-1.617118,1.503298,-0.078287,1.270157,-0.013802,1.745065,-0.474898,0.0,0.587799,...,-0.069683,-0.380559,-0.089286,0.707547,-1.112899,0.855085,-1.130620,2.164060,-0.657599,-0.051718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,-1.824121,0.323996,-0.268218,-0.316258,1.720405,0.785098,0.558624,-0.129661,0.0,0.679439,...,0.059176,1.170102,0.082415,0.530402,-0.947312,0.784439,-0.944206,0.232086,-0.570101,0.173530
488,-1.741798,0.241102,0.428772,0.431650,0.778977,0.785098,1.145971,-0.233692,0.0,0.651898,...,0.055386,1.181682,0.078599,0.405202,0.106567,-1.176969,0.104953,0.627823,-0.565575,0.185181
489,-1.622056,0.185839,1.111241,0.737612,-0.176095,0.785098,1.944763,-0.284327,0.0,0.638493,...,0.062966,1.538369,0.082415,0.575687,0.573926,-1.154401,0.587165,-1.086172,-0.564067,0.189065
490,-1.427475,0.234194,1.633983,1.434527,-0.899221,-0.812702,0.417661,-0.369946,0.0,0.615827,...,0.532920,-0.321497,0.517390,0.856722,0.654398,-1.162251,0.667277,-1.221112,-0.437345,0.515286


In [10]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [11]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [12]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')

# Epasad with 1 cluster (Normal data)

In [13]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'ATT_FLAG']

In [14]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [15]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
fpr = []
cluster_indexes=[]
pca_counts=0
meepc_counts=0

for i,sens in enumerate(sensors):
    print("sensor -> ", sens)
    train_normal = X_normal.loc[:,sens].values
    model = RobustPipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,kscore_init='inertia',tune=False,alpha=0.08,y_truth=y_truth)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    fpr.append(calculate_fpr(y_actual, y_predicted))

    #clustering
    cluster_indexes.append(model.cluster_idx)
    #pca

    pca_counts=max(pca_counts,model.total_pca_attacks)
    
    #meepc
    meepc_counts=max(meepc_counts,model.counts_meepc)
    
    

sensor ->  L_T1
------[CLUSTER] attack points found is 5 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 7 
sensor ->  L_T2
------[CLUSTER] attack points found is 6 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 3 
sensor ->  L_T3
------[CLUSTER] attack points found is 6 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 2 
sensor ->  L_T4
------[CLUSTER] attack points found is 5 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 3 
sensor ->  L_T5
------[CLUSTER] attack points found is 4 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 3 
sensor ->  L_T6
------[CLUSTER] attack points found is 3 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 5 
sensor ->  L_T7
------[CLUSTER] attack points found is 4 
------[PCA] attack points found is 16 
------[Meepc] attack points found is 4 
sensor ->  F_PU1
------[CLUSTER] attack p

In [16]:
attack_idx 

array([418, 419, 420, 421, 430, 431, 432, 442, 443, 444, 445, 446, 462,
       463, 464, 465, 466, 467, 489, 490, 491, 492, 498, 499, 500, 501,
       502, 503, 506, 507, 508, 509, 510, 511], dtype=int64)

In [17]:
cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
percentage = len(commonidxs)/len(attack_idx)*100
print(percentage)
print(commonidxs)



94.11764705882352
[418 419 420 421 430 431 432 442 443 444 445 446 462 463 464 465 466 489
 490 491 492 498 499 500 501 502 503 507 508 509 510 511]


In [18]:
percentage = pca_counts/len(attack_idx)*100
print(percentage)



47.05882352941176


In [19]:
percentage = meepc_counts/len(attack_idx)*100
print(percentage) 

38.23529411764706


In [20]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0])

In [21]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))
print("False Positive Rate ",calculate_fpr(y_actual, y_predicted))

Accuracy  0.8536585365853658
Precision  0.9090909090909091
Recall  0.6666666666666666
F1-score  0.7692307692307692
False Positive Rate  0.038461538461538464


In [21]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.6441525953721075 Median : 0.6341463414634146 Min : 0.6097560975609756 Max : 0.7073170731707317
precision - Mean: 0.2564102564102564 Median : 0.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.029059829059829064 Median : 0.0 Min : 0.0 Max : 0.2
f1 -        Mean: 0.05128273581784019 Median : 0.0 Min : 0.0 Max : 0.33333333333333337


# Multiple clusters + No threshold tuning

In [22]:
cluster_accuracy = []
cluster_precision = []
cluster_recall = []
cluster_fscore = []
cluster_fpr = []
for k in range(2,5):
    print('-------------------------------------',k,'-------------------------------------')
    
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    cluster_indexes=[]
    pca_counts=0
    meepc_counts=0
    for i,sens in enumerate(sensors):
        print(sens)
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = RobustPipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',alpha=0.08,y_truth=y_truth)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted)) 
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual, y_predicted))

        cluster_indexes.append(model.cluster_idx)
        pca_counts=max(pca_counts,model.total_pca_attacks)
        meepc_counts=max(meepc_counts,model.counts_meepc)
    

    #cluster
    cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
    commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
    percentage = len(commonidxs)/len(attack_idx)*100
    print("Cluster - attack points detected {} %".format(percentage))
    #PCA
    
    percentage = pca_counts/len(attack_idx)*100
    print("PCA - attack points detected {} %".format(percentage))
    #Meepc
    percentage = meepc_counts/len(attack_idx)*100
    print("Meepc - attack points detected {} %".format(percentage),"/n")
            
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy.append(accuracy_score(y_actual, y_predicted))
    cluster_precision.append(precision_score(y_actual,y_predicted))
    cluster_recall.append(recall_score(y_actual,y_predicted))
    cluster_fscore.append(f1_score(y_actual,y_predicted))
    cluster_fpr.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy[-1])
    print("Precision ",cluster_precision[-1])
    print("Recall ",cluster_recall[-1])
    print("F1-score ",cluster_fscore[-1])
    print("False Positive Rate ",cluster_fpr[-1],"/n") 
    

------------------------------------- 2 -------------------------------------
L_T1
------[CLUSTER] attack points found is 5 
------[PCA] attack points found is 7 
------[Meepc] attack points found is 6 
------[PCA] attack points found is 9 
------[Meepc] attack points found is 2 
L_T2
------[CLUSTER] attack points found is 6 
------[PCA] attack points found is 8 
------[Meepc] attack points found is 0 
------[PCA] attack points found is 8 
------[Meepc] attack points found is 3 
L_T3
------[CLUSTER] attack points found is 6 
------[PCA] attack points found is 7 
------[Meepc] attack points found is 0 
------[PCA] attack points found is 9 
------[Meepc] attack points found is 0 
L_T4
------[CLUSTER] attack points found is 4 
------[PCA] attack points found is 4 
------[Meepc] attack points found is 1 
------[PCA] attack points found is 12 
------[Meepc] attack points found is 0 
L_T5
------[CLUSTER] attack points found is 4 
------[PCA] attack points found is 10 
------[Meepc] attack po

In [23]:
print("Accuracy Scores: ",cluster_accuracy)
print("Precision Scores: ",cluster_precision)
print("Recall Scores: ",cluster_recall)
print("F1 Scores: ",cluster_fscore)
print("False Positive Rates: ",cluster_fpr)

Accuracy Scores:  [0.8292682926829268, 0.7195121951219512, 0.7317073170731707]
Precision Scores:  [1.0, 0.8181818181818182, 0.9]
Recall Scores:  [0.5333333333333333, 0.3, 0.3]
F1 Scores:  [0.6956521739130436, 0.43902439024390244, 0.45000000000000007]
False Positive Rates:  [0.0, 0.038461538461538464, 0.019230769230769232]
